# Chapter8 조인

### 조인 표현식

스파크는 왼쪽과 오른쪽 데이터셋에 있는 하나 이상의 키값을 비교하고 왼쪽 데이터셋과 오른쪽 데이터셋의 결함 여부를 경정하는 조인 표현식의 평가 결과에 따라 두 개의 데이터셋을 조인한다. 가장 많이 사용하는 조인 표현식은 왼쪽과 오른쪽 데이터셋에 지정된 키가 동일한지 비교하는 동등 조인이다. 

### 조인 타입

* 내부 조인 : 왼쪽과 오른쪽 데이터셋에 키가 있는 로우를 유지
* 외부 조인 : 왼쪽이나 오른쪽 데이터셋에 키가 있는 로우를 유지
* 왼쪽 외부 조인. : 왼쪽 데이터셋에 키가 있는 로우를 유지
* 오른쪽 외부 조인. : 오른쪽 데이터셋에 키가 있는 로우를 유지
* 왼쪽 세미 조인 : 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 있는 경우에는 키가 일치하는 왼쪽 데이터셋만 유지
* 왼쪽 안티 조인 : 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 없는 경우에는 키가 일치하지 않는 왼쪽 데이터셋만 유지
* 자연 조인 : 두 데이터셋에서 동일한 이름을 가진 컬럼을 암시적으로 결합하는 조인
* 교차 조인 또는 카테시안 조인 : 왼쪽 데이터셋의 모든 로우와 오른쪽 데이터셋의 모든 로우를 조합

In [1]:
from pyspark.sql import SparkSession # SparkSession: 스파크 코드를 실행하기 위한 진입점
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])])\
  .toDF("id", "name", "graduate_program", "spark_status")
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")])\
  .toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
  .toDF("id", "status")

In [3]:
person.show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



### 내부 조인

In [7]:
joinExpression = person['graduate_program'] == graduateProgram['id']
joinExpression

Column<b'(graduate_program = id)'>

In [8]:
person.join(graduateProgram,joinExpression).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



* join 메서드의 세 번째 파라키터로 조인타입을 명확하게 지정할 수 있다.

In [9]:
joinType ='inner'

person.join(graduateProgram,joinExpression,joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



### 외부 조인

외부 조인은 DataFrame이나 테이블에 존재하는 키를 평가하여 팜이나 거짓으로 평가한 로우를 포함한다. 왼쪽이나 오른쪽에 DataFrame에 일치하는 로우가 없다면 스파크는 해당 위치에 null을 삽입한다.

In [10]:
joinType ='outer'

person.join(graduateProgram,joinExpression,joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

